In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data and rename columns
df_init = pd.read_csv("dpt2020.csv",delimiter=";")
df_init = df_init.rename(columns={"preusuel": "name", "annais": "year", "nombre": "births"})
df_init = df_init[["year","name","sexe","dpt","births"]]
# delete rows with missing year
df_init = df_init[df_init["year"].str.isnumeric()]
df_init["year"] = df_init["year"].astype(int)
# rename sex numbers (1,2) to string (M,F)
df_init["sexe"] = df_init["sexe"].map({1:"M",2:"F"})
# add gender information with name to avoid aggregating male and female numbers when identical name
df_init["name_old"] = df_init["name"]
df_init["name"] = df_init["name"] + " - " + df_init["sexe"]
# get rid of departments granularity
df_init = df_init.groupby(["year","name","sexe","name_old"])["births"].sum().reset_index()
df_init

,year,name,sexe,name_old,births
0,1900,ABEL - M,M,ABEL,382
1,1900,ABRAHAM - M,M,ABRAHAM,9
2,1900,ACHILLE - M,M,ACHILLE,152
3,1900,ACHILLES - M,M,ACHILLES,4
4,1900,ADAM - M,M,ADAM,9
...,...,...,...,...,...
257583,2020,ÉVA - F,F,ÉVA,156
257584,2020,ÉVAN - M,M,ÉVAN,62
257585,2020,ÉZIO - M,M,ÉZIO,12
257586,2020,ÉZÉCHIEL - M,M,ÉZÉCHIEL,11


In [3]:
# INTEGRATION DES PRENOMS QUI N'APPARAISSENT QU'UNE SEULE FOIS AUX PRENOMS RARES (car pas d'évolution temporelle et pas de calcul de std possible)

# on récupère tous les prénoms qui n'apparaissent qu'une seule fois et on les regroupe par couple année-sexe
df_added_rares = df_init[(1-df_init.duplicated(subset="name",keep=False)).astype(bool)].groupby(["year","sexe"]).sum()["births"]
# on récupère toutes les lignes prénoms rares de la table initiale
df_rares = df_init.loc[df_init["name_old"]=="_PRENOMS_RARES"].set_index(["year","sexe"])["births"]
# on ajoute aux prénoms rares les prénoms apparaissant une seule fois
df_new = df_rares.copy()
df_new.loc[df_added_rares.index] = df_new.loc[df_added_rares.index] + df_added_rares
# on remplace dans la table
df_init.loc[df_init["name_old"]=="_PRENOMS_RARES","births"] = df_new.values
# on supprime les noms présents qu'une fois
df_init = df_init.drop(df_init[(1-df_init.duplicated(subset="name",keep=False)).astype(bool)].index)
df_init

,year,name,sexe,name_old,births
0,1900,ABEL - M,M,ABEL,382
1,1900,ABRAHAM - M,M,ABRAHAM,9
2,1900,ACHILLE - M,M,ACHILLE,152
3,1900,ACHILLES - M,M,ACHILLES,4
4,1900,ADAM - M,M,ADAM,9
...,...,...,...,...,...
257583,2020,ÉVA - F,F,ÉVA,156
257584,2020,ÉVAN - M,M,ÉVAN,62
257585,2020,ÉZIO - M,M,ÉZIO,12
257586,2020,ÉZÉCHIEL - M,M,ÉZÉCHIEL,11


In [4]:
# SAVE AS CSV

df_init.to_csv("names.csv",sep=";",index=False)

In [5]:
# reindex so that every name appears every year

idx = pd.MultiIndex.from_product((set(df_init["year"]),set(df_init["name"])),names=["year","name"])
df = df_init.set_index(["year","name"])["births"].reindex(idx)
df = df.fillna(0).astype(int)
df = df.reset_index(drop = False)
df["sexe"] = df["name"].apply(lambda s : s[-1])
mapping_name_old = df_init[["name","name_old"]].drop_duplicates().set_index("name")["name_old"]
df["name_old"] = df["name"].copy()
df["name_old"] = df["name_old"].map(mapping_name_old)
df = df[["year","name","sexe","name_old","births"]]

# ajout du champ sexe-year
df["sexe_year"] = df["sexe"] + " - " + df["year"].astype(str)
df

,year,name,sexe,name_old,births,sexe_year
0,1900,LUCILIA - F,F,LUCILIA,0,F - 1900
1,1900,LUCINE - F,F,LUCINE,8,F - 1900
2,1900,ROGATIENNE - F,F,ROGATIENNE,0,F - 1900
3,1900,KARINA - F,F,KARINA,0,F - 1900
4,1900,OULFA - F,F,OULFA,0,F - 1900
...,...,...,...,...,...,...
1492409,2020,MARDEMOUTOU - M,M,MARDEMOUTOU,0,M - 2020
1492410,2020,ERELL - F,F,ERELL,28,F - 2020
1492411,2020,ROSIE - F,F,ROSIE,81,F - 2020
1492412,2020,JIHÈNE - F,F,JIHÈNE,0,F - 2020


In [6]:
df_init["births"].sum() == df["births"].sum()

True

In [21]:
# SAVE AS CSV

df.to_csv("names_reindexed.csv",sep=";",index=False)

Pour tester les valeurs obtenues sur tableau :

In [41]:
df_init.loc[(df_init["name"]=="JULIEN - M") & (df_init["year"]>=1969),"births"].sum()

287905

In [42]:
df.loc[(df["year"]>=1969)  & (df["sexe"]=="M"),"births"].sum()

18344056

In [43]:
287905/18344056

0.015694729671562276